# Markowitz Analysis on Eurostat Data
Ellie Cox

This file will conduct markowitz and black-litterman analysis on energy supply data retrieved from eurostat. 
First with the full data, then taking the average when excluding one country at a time, the average when excluding one year of data at a time, and lastly using a randomly selected 75\% of the data

## Read Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypfopt as pyp
import math
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
import cvxopt as opt
from cvxopt import blas, solvers
import random
import pickle
solvers.options['show_progress'] = False

## Read Data

In [ ]:
# Read data using pandas
data = pd.read_csv("/Users/elizabeth/Documents/Master's Project/Data/EU_TotalEnergySupply.csv")

# Create list of column names
data.columns.values.tolist()
# Rename columns to make life easier
data.columns = [c.replace(' ', '_') for c in data.columns] # remove spaces
data.columns = [c.replace('(', '') for c in data.columns] # remove open parenthesis
data.columns = [c.replace(')', '') for c in data.columns] # remove close parenthesis
data.columns.values.tolist()

# Get rid of ':' and shorten other names
data = data.replace([':'],'')
data = data.replace(['European Union - 27 countries (from 2020)'],'EU')
data = data.replace(['Euro area - 19 countries  (from 2015)'],'Euro area')
data = data.replace(['Germany (until 1990 former territory of the FRG)'],'Germany')
data = data.replace(['Kosovo (under United Nations Security Council Resolution 1244/99)'],'Kosovo')

# Change Data type to numeric
data[data.columns[2:]] = data[data.columns[2:]].apply(pd.to_numeric, errors ='coerce')

## Make an optimizing function

In [ ]:
def markowitz(ret_mat):
    '''
    Calculates the markowitz optimal portfolio weights, returns, and risks for one set of return data
    returns are of shape: n_obs x n_assets
    '''
    n = len(ret_mat.T) # n assets
    return_vec = np.array(ret_mat).T
    
    N = len(ret_mat) # n obs
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)] #[(1/n) + t/N for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(return_vec))
    pbar = opt.matrix(np.mean(return_vec, axis = 1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n)) # negative nxn identity matrix
    h = opt.matrix(0.0, (n,1))
    A = opt.matrix(1.0, (1,n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    ## Calculate risk and returns for frontier
    ret = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## Calculate the 2nd degree polynomail of the frontier curve
    m1 = np.polyfit(ret, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    ## Calculate the optimal portfolio
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    #ret = np.zeros((n,1))
    #rsk = np.zeros((n,1))
    #for j in range(n):
    #    ret[j] = np.sum(wt[j] * return_vec[:,j])
    #    rsk[j] = wt[j] * np.diagonal(np.cov(return_vec))[j] * wt[j]
    return np.asarray(wt)#, ret, rsk

## Imputed Data
Missing Data is handled in 2 ways:

    1) Replacing missing data with the country's average
    2) Dropping it

In [ ]:
## 1) Impute data with the average
frames = []
for i in list(set(data['Country'])):
            df_country = data[data['Country'] == i] 
            df_country['Total_GWH'].fillna(df_country['Total_GWH'].mean(),inplace = True)
            df_country['Solid_fossil_fuels'].fillna(df_country['Solid_fossil_fuels'].mean(), inplace = True)
            df_country['Peat_and_peat_products'].fillna(df_country['Peat_and_peat_products'].mean(), inplace = True)
            df_country['Solar_Thermal'].fillna(df_country['Solar_Thermal'].mean(), inplace = True)
            df_country['Oil_and_petroleum_products'].fillna(df_country['Oil_and_petroleum_products'].mean(), inplace = True)
            df_country['Natural_gas'].fillna(df_country['Natural_gas'].mean(), inplace = True)
            df_country['Renewables_and_biofuels'].fillna(df_country['Renewables_and_biofuels'].mean(), inplace = True)
            df_country['Nuclear_heat'].fillna(df_country['Nuclear_heat'].mean(),inplace = True)
            df_country['Hydro'].fillna(df_country['Hydro'].mean(),inplace = True)
            df_country['Geothermal'].fillna(df_country['Geothermal'].mean(),inplace = True)
            df_country['Ambient_Heat'].fillna(df_country['Ambient_Heat'].mean(),inplace = True)
            df_country['Tide_wave_and_ocean'].fillna(df_country['Tide_wave_and_ocean'].mean(),inplace = True)
            df_country['Wind'].fillna(df_country['Wind'].mean(),inplace = True)
            df_country['Biofuels_solid'].fillna(df_country['Biofuels_solid'].mean(),inplace = True)
            df_country['Biofuels_other'].fillna(df_country['Biofuels_other'].mean(),inplace = True)
            df_country['Biofuels'].fillna(df_country['Biofuels'].mean(),inplace = True)
            frames.append(df_country)
            final_df = pd.concat(frames)
#final_df[(final_df.Country == 'United Kingdom') & (final_df.Year == 2020)]
data_impute = final_df

In [ ]:
cdat = data_impute.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Wind', 'Biofuels_solid', 'Biofuels_other']]

Full data

In [ ]:
n = len(cdat.T) # n assets
returns = np.array(cdat).T
    
N = len(cdat) # n obs
mus = [10**(5.0 * t/N - 1.0) for t in range(N)]#[(1/n) + t/N for t in range(N)]#
    
# Convert to cvxopt matrices
S = opt.matrix(np.cov(returns))
pbar = opt.matrix(np.mean(returns, axis = 1))
    
# Create constraint matrices
G = -opt.matrix(np.eye(n)) # negative nxn identity matrix
h = opt.matrix(0.0, (n,1))
A = opt.matrix(1.0, (1,n))
b = opt.matrix(1.0)
    
# Calculate efficient frontier weights using quadratic programming
portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x']
                  for mu in mus]
## Calculate risk and returns for frontier
ret = [blas.dot(pbar, x) for x in portfolios]
risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
## Calculate the 2nd degree polynomail of the frontier curve
m1 = np.polyfit(ret, risks, 2)
x1 = np.sqrt(m1[2] / m1[0])
## Calculate the optimal portfolio
wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
ret = np.zeros((n,1))
rsk = np.zeros((n,1))
for j in range(n):
    ret[j] = 1/n * np.sum(wt[j] * np.asmatrix(returns)[:,j])
    rsk[j] = 1/N * np.sum(1/n * np.sum(np.asmatrix(risks)[:,j]))


Exclude one country at a time

In [ ]:
cdat = data_impute.loc[:,['Country','Year','Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]

In [ ]:
weight_country = np.zeros((len(data_impute.groupby('Country')),len(cdat.T)-2))
return_res = np.zeros((len(data_impute.groupby('Country')),len(cdat.T)-2))
risks_res = np.zeros((len(data_impute.groupby('Country')),len(cdat.T)-2))

for i in range(len(data_impute.groupby('Country'))):
    country_dat = cdat.loc[(cdat.Country != cdat.Country.unique()[i])]
    country_dat = country_dat.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]
    weights = markowitz(country_dat)
    weight_country[i,:] = weights.T
    return_res[i,:] = np.array(returns).T
    risks_res[i,:] = np.array(risks).T

Exclude one year at a time

In [ ]:
weight_year = np.zeros((len(data_impute.groupby('Year')),len(cdat.T)-2))
return_res = np.zeros((len(data_impute.groupby('Year')),len(cdat.T)-2))
risks_res = np.zeros((len(data_impute.groupby('Year')),len(cdat.T)-2))

for i in range(len(data_impute.groupby('Year'))):
    year_dat = cdat.loc[(cdat.Year != cdat.Year.unique()[i])]
    year_dat = year_dat.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]
    weights = markowitz(year_dat)
    weight_year[i,:] = weights.T
    return_res[i,:] = np.array(returns).T
    risks_res[i,:] = np.array(risks).T

Drop random 20% of data

In [ ]:
weight = np.zeros((100,len(cdat.T)))
return_res = np.zeros((100,len(cdat.T)))
risks_res = np.zeros((100,len(cdat.T)))

for i in range(100):
    
    # Randomly take out 20% of the data
    _80_perct = int(cdat.shape[0]*4/5)
    cdat = cdat.iloc[random.sample(list(range(cdat.shape[0])), _80_perct)]
    data = cdat.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]
    
    weights = markowitz(data)
    weight[i,:] = weights.T
    return_res[i,:] = np.array(returns).T
    risks_res[i,:] = np.array(risks).T

## Dropped data

In [ ]:
data_drop = data.dropna()

In [ ]:
cdat = data_drop.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]


Full data

In [ ]:
n = len(cdat.T) # n assets
returns = np.array(cdat).T
    
N = len(cdat) # n obs
mus = [(1/n) + t/N for t in range(N)]#[np.mean(returns, axis=0)]#[10**(5.0 * t/N - 1.0) for t in range(N)]
    
# Convert to cvxopt matrices
S = opt.matrix(np.cov(returns))
pbar = opt.matrix(np.mean(returns, axis = 1))
    
# Create constraint matrices
G = -opt.matrix(np.eye(n)) # negative nxn identity matrix
h = opt.matrix(0.0, (n,1))
A = opt.matrix(1.0, (1,n))
b = opt.matrix(1.0)
    
# Calculate efficient frontier weights using quadratic programming
portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x']
                  for mu in mus]
## Calculate risk and returns for frontier
ret = [blas.dot(pbar, x) for x in portfolios]
risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
## Calculate the 2nd degree polynomail of the frontier curve
m1 = np.polyfit(ret, risks, 2)
x1 = np.sqrt(m1[2] / m1[0])
## Calculate the optimal portfolio
weight_full_drop = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
ret = np.zeros((n,1))
rsk = np.zeros((n,1))
for j in range(n):
    ret[j] = np.sum(np.array(wt)[j] * np.mean(returns, axis = 1))
    rsk = wt.T @ np.cov(returns) @ wt

Drop one country at a time

In [ ]:
cdat = data_drop.loc[:,['Country','Year','Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]

In [ ]:
weight_country_drop = np.zeros((len(data.groupby('Country')),len(cdat.T)-2))
return_res = np.zeros((len(data.groupby('Country')),len(cdat.T)-2))
risks_res = np.zeros((len(data.groupby('Country')),len(cdat.T)-2))

for i in range(len(data.groupby('Country'))):
    country_dat = cdat.loc[(cdat.Country != cdat.Country.unique()[i])]
    country_dat = country_dat.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]
    weights = markowitz(country_dat)
    weight_country_drop[i,:] = weights.T
    return_res[i,:] = np.array(returns).T
    risks_res[i,:] = np.array(risks).T

Drop one year at a time

In [ ]:
weight_year_drop = np.zeros((len(data.groupby('Year')),len(cdat.T)-2))
return_res = np.zeros((len(data.groupby('Year')),len(cdat.T)-2))
risks_res = np.zeros((len(data.groupby('Year')),len(cdat.T)-2))

for i in range(len(data.groupby('Year'))):
    year_dat = cdat.loc[(cdat.Year != cdat.Year.unique()[i])]
    year_dat = year_dat.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]
    weights = markowitz(year_dat)
    weight_year_drop[i,:] = weights.T
    return_res[i,:] = np.array(returns).T
    risks_res[i,:] = np.array(risks).T

Drop random 20% of data

In [ ]:
weight = np.zeros((100,len(cdat.T)))
return_res = np.zeros((100,len(cdat.T)))
risks_res = np.zeros((100,len(cdat.T)))

for i in range(100):
    
    # Randomly take out 20% of the data
    _80_perct = int(cdat.shape[0]*4/5)
    cdat = cdat.iloc[random.sample(list(range(cdat.shape[0])), _80_perct)]
    data = cdat.loc[:,['Solid_fossil_fuels','Peat_and_peat_products','Oil_and_petroleum_products', 
                          'Natural_gas', 'Nuclear_heat', 'Hydro', 'Solar_Thermal', 'Geothermal', 
                          'Ambient_Heat', 'Tide_wave_and_ocean', 'Biofuels_solid', 'Biofuels_other']]
    
    weights = markowitz(data)
    weight[i,:] = weights.T
    #return_res[i,:] = np.array(returns).T
    #risks_res[i,:] = np.array(risks).T

## Save results

In [ ]:
MarkowitzResults_Weights = [weight_full, weight_country, weight_year, weight_rand, 
                              weight_full_drop, weight_country_drop, weight_year_drop, weight_rand_drop]

file_name = "EUMarkowitzResultsWeights.pkl"
open_file = open(file_name, "wb")
pickle.dump(MarkowitzResults_Weights, open_file)
open_file.close()